In [1]:
%load_ext autoreload
%autoreload 2

import os, sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

# Classic Text Retrieval with SQL

In [2]:
import ipywidgets as widgets
from typing import Callable

global nDocs, index, vocabulary, documents
DEBUG = False

In [4]:
import sqlite3

db = sqlite3.connect(":memory:")
db.execute("CREATE TABLE document(docID INTEGER PRIMARY KEY AUTOINCREMENT, title, year, runtime, rating, genre, actors, summary)")
db.execute("CREATE TABLE term(term TEXT, df INTEGER, idf REAL)")
db.execute("CREATE TABLE posting(term TEXT, docID INTEGER, tf INTEGER)")
db.execute("CREATE INDEX inverted_list ON posting(term, docID)")

In [10]:
data[1]

{'title': 'The Godfather',
 'year': 1972,
 'runtime': 175,
 'rating': 9.2,
 'genre': 'Crime Drama',
 'actors': 'Marlon Brando Al Pacino James Caan Diane Keaton',
 'summary': "An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son."}

In [ ]:
from utils import stopwords
from itertools import groupby
import re

def tokenize(text: str) -> list[str]:
    text = re.sub(r'[,\.\-\?!\(\)\s:;_\'"\+\*\&\$]', ' ', text.lower())
    text = re.sub(r'\s+', ' ', text).strip()
    return filter(lambda t: t not in stopwords.english, text.split(' '))

def bag_of_words(tokens: list[str]) -> dict[str, int]:
    return dict([(token, len(list(group))) for token, group in groupby(sorted(tokens))])

def build_index_vsm(collection: list[dict]) -> None:
    global nDocs, index, documents, vocabulary
    nDocs = 0
    index = {}
    documents = {}
    vocabulary = {}
    for doc in collection:
        nDocs += 1
        # add document to documents dict, with id as key and all properties
        doc_id = doc['id'] = nDocs
        documents[doc_id] = doc
        # get terms from all string properties of doc
        text = ' '.join([value for key, value in doc.items() if type(value) == str])
        doc['vector'] = bag_of_words(tokenize(text))
        doc['len'] = sum([tf for _, tf in doc['vector'].items()])
        # update vocabulary (holding df values during this loop) and index (term -> postings)
        for term, tf in doc['vector'].items():
            vocabulary[term] = vocabulary.get(term, 0) + 1
            index[term] = index.get(term, []) + [(doc_id, tf)]
    # update vocabulary -> create dict with df, idf, and idf_bm25 values
    vocabulary = dict([(term, {'df': df, 'idf': idf(df, nDocs), 'idf_bm25': idf_bm25(df, nDocs)}) for term, df in vocabulary.items()])
    # calculate norm of vector for cosine measure
    for doc in documents.values():
        doc['norm'] = sum([(tf * vocabulary[term]['idf']) ** 2 for term, tf in doc['vector'].items()]) ** 0.5      

In [8]:
# loading the imdb data set (1000 movies)
from datasets.docs import imdb
from utils import stopwords

data = imdb.load()
def doc_format_imdb(doc: dict) -> str:
    trim = lambda s,n: len(s) > n and s[:n] + "\u2026" or s
    title_ex = '{title_short} ({year}, {runtime}m, {rating})'.format(title_short=trim(doc['title'], 30), **doc)
    return '{title_ex:<50} {genre_short:<20} {summary} [{actors}]'.format(title_ex=title_ex, genre_short=trim(doc['genre'], 18), **doc)

for item in data[:5]:
    print(doc_format_imdb(item))

data[0]

The Shawshank Redemption (1994, 142m, 9.3)         Drama                Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency. [Tim Robbins Morgan Freeman Bob Gunton William Sadler]
The Godfather (1972, 175m, 9.2)                    Crime Drama          An organized crime dynasty's aging patriarch transfers control of his clandestine empire to his reluctant son. [Marlon Brando Al Pacino James Caan Diane Keaton]
The Dark Knight (2008, 152m, 9.0)                  Action Crime Drama   When the menace known as the Joker wreaks havoc and chaos on the people of Gotham, Batman must accept one of the greatest psychological and physical tests of his ability to fight injustice. [Christian Bale Heath Ledger Aaron Eckhart Michael Caine]
The Godfather: Part II (1974, 202m, 9.0)           Crime Drama          The early life and career of Vito Corleone in 1920s New York City is portrayed, while his son, Michael, expands and tightens his 

{'title': 'The Shawshank Redemption',
 'year': 1994,
 'runtime': 142,
 'rating': 9.3,
 'genre': 'Drama',
 'actors': 'Tim Robbins Morgan Freeman Bob Gunton William Sadler',
 'summary': 'Two imprisoned men bond over a number of years, finding solace and eventual redemption through acts of common decency.'}

In [ ]:
build_index(data)
print('nDocs = {nDocs}, nTerms = {nTerms}, nPostings'.format(nDocs=nDocs, nTerms=len(vocabulary), nPostings=sum([len(postings) for postings in index.values()])))

In [ ]:
# print vocabulary with df and idf
print('term               df     idf idf_bm25    postings')
print('-' * 100)
for term, item in sorted(vocabulary.items(), key=lambda t: -t[1]['df'])[:10]:
    print("{term:16} {df:>4d} {idf:>7.2f} {idf_bm25:>8.2f}    {postings}".format(term=term.ljust(10), df=item['df'], idf=item['idf'], idf_bm25=item['idf_bm25'], postings=index[term]))
print()
for term, item in random.sample(list(vocabulary.items()), 15):
    print("{term:16} {df:>4d} {idf:>7.2f} {idf_bm25:>8.2f}    {postings}".format(term=term.ljust(10), df=item['df'], idf=item['idf'], idf_bm25=item['idf_bm25'], postings=index[term]))

In [ ]:
# print a few documents
print('  id |  len     norm   vector')
print('-' * 100)
for doc_id in random.sample(range(1, len(documents) + 1), 25):
    print("{id:>4} | {len:>4d} {norm:>8.2f}   {vector}".format(id=doc_id, len=documents[doc_id]['len'], norm=documents[doc_id]['norm'], vector=str(documents[doc_id]['vector'])))

In [ ]:
from typing import Callable
def search(query: str, k: int, measure: str, predicate: str, algorithm: str) -> TopKList:
    query_vector = bag_of_words(tokenize(query))
    vsmodel = {'ddat': VSModel_DAAT}.get(algorithm.lower(), VSModel_DAAT)
    predicate_func = {
        'year > 1990 (predicate)': lambda doc_id: documents[doc_id]['year'] > 1990, 
        'year <= 1990 (predicate)': lambda doc_id: documents[doc_id]['year'] <= 1990, 
    }.get(predicate, None)
    selected_docs = {
        'top-100 (selection)': set(range(1,101))
    }.get(predicate, None)
    topk = vsmodel.query(query_vector, k, measure, predicate=predicate_func, selected_docs=selected_docs)
    print_topk(topk, doc_format_imdb)

# options for the dialog
queries = ['star wars', 'drama morgan freeman', 'comedy']
measures = ['cosine', 'dot', 'bm25']
predicates = ['<none>', 'year > 1990 (predicate)', 'year <= 1990 (predicate)', 'top-100 (selection)']
algorithm = ['ddat', 'taat']

# interactive selection of scenario
widgets.interact(search, 
    query=widgets.Dropdown(options=queries), 
    k=widgets.IntSlider(min=5, max=50, step=5, value=20),
    measure=widgets.Dropdown(options=measures),
    predicate=widgets.Dropdown(options=predicates),
    algorithm=widgets.Dropdown(options=algorithm),
);


In [4]:
con = sqlite3.connect(":memory:")
cur = con.execute("CREATE TABLE lang(name, first_appeared)")

# This is the named style used with executemany():
data = (
    {"name": "C", "year": 1972},
    {"name": "Fortran", "year": 1957},
    {"name": "Python", "year": 1991},
    {"name": "Go", "year": 2009},
)
cur.executemany("INSERT INTO lang VALUES(:name, :year)", data)
cur.execute("CREATE INDEX year ON lang(first_appeared)")
# This is the qmark style used in a SELECT query:
params = (1972,)
cur.execute("SELECT * FROM lang WHERE first_appeared = ?", params)
print(cur.fetchall())

[('C', 1972)]


```
SELECT a.DocID
  FROM Index a,Index b,Index c  
 WHERE a.Term=‘vehicle‘ AND
       b.Term=‘sales‘ AND
       c.Term=‘italy‘ AND
       a.DocID=b.DocID AND
       a.DocID=c.DocID;

DELETE FROM Query;
INSERT INTO Query    VALUES(‘vehicle‘,1);
INSERT INTO Query    VALUES(‘sales‘,1);
INSERT INTO Query    VALUES(‘italy‘,1);

SELECT i.DocID
  FROM Index i, Query q
 WHERE i.Term=q.Term
 GROUP BY i.DocID
HAVING COUNT(i.Term)=
    (SELECT COUNT(*) FROM QUERY)


	DELETE FROM Query;
	INSERT INTO Query	    VALUES(‘vehicle‘,1);
	INSERT INTO Query     VALUES(‘sales‘,1);
	INSERT INTO Query	    VALUES(‘italy‘,1);


	SELECT i.DocID, SUM(q.tf * t.idf * i.tf * t.idf)
	  FROM Query q, Index i, Term t
	 WHERE q.Term=t.Term AND
	       i.Term=t.Term
	 GROUP BY i.DocID
	 ORDER BY 2 DESC;

```